In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:99%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

# 인코더 LSTM과 디코더 LSTM으로 스마트 번역기 만들기

In [2]:
# 1. 패키지 수입
import numpy as np
import pandas as pd
from time import time

from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼 파라미터 (정확도나 속도에 영향을 미치는 값)
MY_HIDDEN = 128
MY_EPOCH  = 500

In [3]:
# 2. 번역 데이터 불러오기
raw = pd.read_csv('trans_dataset/translate.csv', header=None)
eng_kor = raw.values.tolist() # 데이터 프레임을 list로 변환
print(len(eng_kor))
print(eng_kor[:3])

110
[['cold', '감기'], ['come', '오다'], ['cook', '요리']]


In [4]:
# 3. 영어알파벳과 한글알파벳 리스트 만들기

e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
# {c:i for i, c in enumerate(e_alpha)}
k_alpha = pd.read_csv('trans_dataset/korean.csv', header=None)[0].tolist()
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 : ', alpha)
alpha_total_size = len(alpha)
print('전체 알파벳 갯수(원핫인코딩할 size) :', alpha_total_size)
print('영어 알파벳 갯수 :', len(e_alpha))
print('한글 글자수 갯수 :', len(k_alpha))

영어와 한글 알파벳 :  ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩할 size) : 171
영어 알파벳 갯수 : 29
한글 글자수 갯수 : 142


In [5]:
# 4. 문자당 num를 갖는 dict 만들기
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [6]:
eng_kor[0]
print(char_to_num['c'], char_to_num['o'], char_to_num['l'], char_to_num['d'])
print([char_to_num[c] for c in eng_kor[0][0]])
print(char_to_num['감'], char_to_num['기'])
print([char_to_num[c] for c in eng_kor[0][1]])

5 17 14 6
[5, 17, 14, 6]
32 46
[32, 46]


In [7]:
# 원핫인코딩 방법1 : 이 코드에서는 적용 힘듦. display(pd.get_dummies([5, 3, 6]))
# 원핫인코딩 방법2
to_categorical([5, 3, 6], num_classes=10) # 0~9인덱스 원핫인코딩

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [8]:
# 원핫인코딩 방법3 : 단위행렬을 이용한 방법
np.eye(10)[[5,3,6]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [9]:
# 5. 인코더 입력(영어알파벳을 숫자로 바꾼 값을 원핫인코딩), 
#    디코더 입력("S한글"을 숫자로 바꾼 값을 원핫인코딩)
#    디코더 출력("한글E"을 숫자로 바꾼 값)
def encoding(eng_kor=eng_kor):
    enc_in = [] 
    dec_in = [] 
    dec_out = []
    for data in eng_kor:
        # 인코딩 입력 데이터(영어->숫자->원핫인코딩)
        eng = [char_to_num[c] for c in data[0]]
        eng_one = np.eye(alpha_total_size)[eng]
        # eng_one = to_categorical(eng, num_classes=alpha_total_size)
        enc_in.append(eng_one)
        # 디코더 입력 데이터("S한글"->숫자->원핫인코딩)
        kor = [char_to_num[c] for c in 'S'+data[1]]
        kor_one = np.eye(alpha_total_size)[kor]
        dec_in.append(kor_one)
        # 디코더 출력 데이터 ("한글E"->숫자)
        kor = [char_to_num[c] for c in data[1]+'E']
        dec_out.append(kor)
    return enc_in, dec_in, dec_out

In [10]:
#sample = [['wood','나무'],['word','단어']]
#encoding(sample)

In [13]:
# 6. 전체 번역 데이터 처리
X_enc, X_dec, Y_dec = encoding(eng_kor)

# numpy로 전환
X_enc = np.array(X_enc)
X_dec = np.array(X_dec)
Y_dec = np.array(Y_dec).reshape(-1, 3, 1) # 차원 추가

# 최종 입출력 데이터 shape 확인
X_enc.shape, X_dec.shape, Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3, 1))

In [16]:
# 7. 모델 구현
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))
# 출력, state_c, state_h
_, state_h, state_c = LSTM(units=MY_HIDDEN, # 128
                          return_state=True)(ENC_IN)
# 인코더와 디코더 연결
link = [state_h, state_c]

# 디코더 구현
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN,
              return_sequences=True)(DEC_IN, initial_state=link)
# 최종 출력층
DEC_OUT = Dense(units=alpha_total_size,
               activation='softmax')(DEC_MID)

# 모델 구조 구현
model = Model(inputs=[ENC_IN, DEC_IN],
             outputs=DEC_OUT)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 128),        153600      ['input_3[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                            

In [17]:
# 8. 학습과정 설정 & 학습하기
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])
begin = time()
model.fit([X_enc, X_dec],
          Y_dec,
          epochs=MY_EPOCH,
          verbose=1)
end = time()
print('학습시간 :', end-begin)

Epoch 1/500
4/4 [==============================] - 2s 14ms/step - loss: 5.1133 - accuracy: 0.2061
Epoch 2/500
4/4 [==============================] - 0s 14ms/step - loss: 4.9186 - accuracy: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 19ms/step - loss: 3.9623 - accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 15ms/step - loss: 3.4514 - accuracy: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 3ms/step - loss: 3.4032 - accuracy: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 10ms/step - loss: 3.3659 - accuracy: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 9ms/step - loss: 3.3396 - accuracy: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 5ms/step - loss: 3.3207 - accuracy: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 5ms/step - loss: 3.2950 - accuracy: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 5ms/step - loss: 3.2669 - accuracy: 0.3333
Epoch 11/500
4/4 [======

4/4 [==============================] - 0s 7ms/step - loss: 1.2891 - accuracy: 0.8455
Epoch 85/500
4/4 [==============================] - 0s 7ms/step - loss: 1.2979 - accuracy: 0.8242
Epoch 86/500
4/4 [==============================] - 0s 9ms/step - loss: 1.2886 - accuracy: 0.8545
Epoch 87/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1746 - accuracy: 0.8818
Epoch 88/500
4/4 [==============================] - 0s 6ms/step - loss: 1.1476 - accuracy: 0.8727
Epoch 89/500
4/4 [==============================] - 0s 6ms/step - loss: 1.1124 - accuracy: 0.8939
Epoch 90/500
4/4 [==============================] - 0s 6ms/step - loss: 1.0795 - accuracy: 0.9091
Epoch 91/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0454 - accuracy: 0.9121
Epoch 92/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0831 - accuracy: 0.8788
Epoch 93/500
4/4 [==============================] - 0s 10ms/step - loss: 0.9896 - accuracy: 0.9273
Epoch 94/500
4/4 [==============

Epoch 167/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0374 - accuracy: 1.0000
Epoch 168/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0370 - accuracy: 1.0000
Epoch 169/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0339 - accuracy: 1.0000
Epoch 170/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0333 - accuracy: 1.0000
Epoch 171/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0308 - accuracy: 1.0000
Epoch 172/500
4/4 [==============================] - 0s 4ms/step - loss: 0.0288 - accuracy: 1.0000
Epoch 173/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0265 - accuracy: 1.0000
Epoch 174/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0260 - accuracy: 1.0000
Epoch 175/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0255 - accuracy: 1.0000
Epoch 176/500
4/4 [==============================] - 0s 11ms/step - loss: 0.0237 - accuracy: 1.0000
Epoch 17

4/4 [==============================] - 0s 6ms/step - loss: 2.8208e-04 - accuracy: 1.0000
Epoch 249/500
4/4 [==============================] - 0s 6ms/step - loss: 2.6209e-04 - accuracy: 1.0000
Epoch 250/500
4/4 [==============================] - 0s 11ms/step - loss: 2.6711e-04 - accuracy: 1.0000
Epoch 251/500
4/4 [==============================] - 0s 6ms/step - loss: 2.2834e-04 - accuracy: 1.0000
Epoch 252/500
4/4 [==============================] - 0s 6ms/step - loss: 2.0917e-04 - accuracy: 1.0000
Epoch 253/500
4/4 [==============================] - 0s 11ms/step - loss: 2.0750e-04 - accuracy: 1.0000
Epoch 254/500
4/4 [==============================] - 0s 11ms/step - loss: 1.8520e-04 - accuracy: 1.0000
Epoch 255/500
4/4 [==============================] - 0s 11ms/step - loss: 1.7800e-04 - accuracy: 1.0000
Epoch 256/500
4/4 [==============================] - 0s 6ms/step - loss: 3.0158e-04 - accuracy: 1.0000
Epoch 257/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0036 - a

4/4 [==============================] - 0s 6ms/step - loss: 5.8391e-06 - accuracy: 1.0000
Epoch 328/500
4/4 [==============================] - 0s 6ms/step - loss: 5.6364e-06 - accuracy: 1.0000
Epoch 329/500
4/4 [==============================] - 0s 11ms/step - loss: 5.3915e-06 - accuracy: 1.0000
Epoch 330/500
4/4 [==============================] - 0s 6ms/step - loss: 5.2513e-06 - accuracy: 1.0000
Epoch 331/500
4/4 [==============================] - 0s 4ms/step - loss: 5.0302e-06 - accuracy: 1.0000
Epoch 332/500
4/4 [==============================] - 0s 4ms/step - loss: 4.9457e-06 - accuracy: 1.0000
Epoch 333/500
4/4 [==============================] - 0s 6ms/step - loss: 4.8168e-06 - accuracy: 1.0000
Epoch 334/500
4/4 [==============================] - 0s 5ms/step - loss: 4.6087e-06 - accuracy: 1.0000
Epoch 335/500
4/4 [==============================] - 0s 7ms/step - loss: 4.5314e-06 - accuracy: 1.0000
Epoch 336/500
4/4 [==============================] - 0s 10ms/step - loss: 4.4071e-06 -

4/4 [==============================] - 0s 7ms/step - loss: 1.2065e-06 - accuracy: 1.0000
Epoch 407/500
4/4 [==============================] - 0s 10ms/step - loss: 1.1906e-06 - accuracy: 1.0000
Epoch 408/500
4/4 [==============================] - 0s 11ms/step - loss: 1.1856e-06 - accuracy: 1.0000
Epoch 409/500
4/4 [==============================] - 0s 6ms/step - loss: 1.1621e-06 - accuracy: 1.0000
Epoch 410/500
4/4 [==============================] - 0s 3ms/step - loss: 1.1524e-06 - accuracy: 1.0000
Epoch 411/500
4/4 [==============================] - 0s 6ms/step - loss: 1.1375e-06 - accuracy: 1.0000
Epoch 412/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1220e-06 - accuracy: 1.0000
Epoch 413/500
4/4 [==============================] - 0s 6ms/step - loss: 1.1184e-06 - accuracy: 1.0000
Epoch 414/500
4/4 [==============================] - 0s 7ms/step - loss: 1.1043e-06 - accuracy: 1.0000
Epoch 415/500
4/4 [==============================] - 0s 11ms/step - loss: 1.0989e-06 

4/4 [==============================] - 0s 4ms/step - loss: 6.2458e-07 - accuracy: 1.0000
Epoch 486/500
4/4 [==============================] - 0s 5ms/step - loss: 6.2278e-07 - accuracy: 1.0000
Epoch 487/500
4/4 [==============================] - 0s 5ms/step - loss: 6.1483e-07 - accuracy: 1.0000
Epoch 488/500
4/4 [==============================] - 0s 6ms/step - loss: 6.1519e-07 - accuracy: 1.0000
Epoch 489/500
4/4 [==============================] - 0s 6ms/step - loss: 6.1050e-07 - accuracy: 1.0000
Epoch 490/500
4/4 [==============================] - 0s 11ms/step - loss: 6.0941e-07 - accuracy: 1.0000
Epoch 491/500
4/4 [==============================] - 0s 11ms/step - loss: 6.0291e-07 - accuracy: 1.0000
Epoch 492/500
4/4 [==============================] - 0s 3ms/step - loss: 5.9749e-07 - accuracy: 1.0000
Epoch 493/500
4/4 [==============================] - 0s 4ms/step - loss: 5.9894e-07 - accuracy: 1.0000
Epoch 494/500
4/4 [==============================] - 0s 6ms/step - loss: 5.9894e-07 -

In [19]:
# 9. 모델 사용
test = [['cold', 'PP'],
        ['down', 'PP'],
        ['desk', 'PP'],
        ['love', 'PP'],
        ['olve', 'PP'],
        ['lvoe', 'PP'],
        ['loev', 'PP'],
        ['eovl', 'PP']]
enc_in, dec_in, dec_out = encoding(test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
enc_in.shape, dec_in.shape

((8, 4, 171), (8, 3, 171))

In [20]:
pred = model.predict([enc_in, dec_in])
pred

1/1 [==============================] - 1s 555ms/step


array([[[1.45521226e-14, 2.13867666e-16, 2.28490322e-14, ...,
         6.19955693e-13, 1.06411848e-13, 7.35619494e-08],
        [6.07977380e-15, 2.40125320e-09, 9.70611666e-15, ...,
         2.03358164e-11, 1.62863938e-15, 1.83702864e-09],
        [8.26578160e-17, 9.99999881e-01, 1.28392182e-16, ...,
         3.47208807e-12, 1.68246303e-17, 1.35090994e-13]],

       [[4.94168169e-14, 3.93200900e-17, 1.26091078e-14, ...,
         5.65465177e-15, 4.05440384e-13, 4.36312630e-09],
        [4.83158977e-13, 9.90715954e-10, 8.10618459e-14, ...,
         9.59803972e-11, 4.43152391e-14, 1.95301501e-12],
        [4.89455077e-16, 9.99999642e-01, 4.30160124e-16, ...,
         4.28972136e-12, 7.83385767e-17, 1.23350299e-14]],

       [[4.34528173e-14, 1.18921811e-16, 3.76321153e-14, ...,
         1.21389583e-15, 6.40644343e-11, 1.23340422e-12],
        [6.18029729e-14, 7.38993450e-11, 4.08271507e-14, ...,
         1.63599318e-13, 8.84198246e-14, 6.94896004e-14],
        [1.76606179e-15, 9.99999762e

In [21]:
pred.shape

(8, 3, 171)

In [27]:
# alpha[32], char_to_num['감']
# 출력형식 : cold -> 감기(32, xx, 1)
for i in range(len(pred)):
    eng = test[i][0]
    hat = pred[i].argmax(axis=1)
    kor = alpha[hat[0]] + alpha[hat[1]]
    print("{} -> {}({})".format(eng, kor, hat))

cold -> 감기([32 46  1])
down -> 아래([111  67   1])
desk -> 책상([149  97   1])
love -> 사랑([96 66  1])
olve -> 사랑([96 66  1])
lvoe -> 사랑([96 66  1])
loev -> 사랑([96 66  1])
eovl -> 도리([62 72  1])
